In [ ]:
# Install and import packages
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
from IPython.display import JSON

# Data visualization packages
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Natual language processing packages
# Natural Language Toolkit
import nltk
# Stop words are words that you want to ignore, and can be filtered out
from nltk.corpus import stopwords
# Tokenizing allows you to split up text by word or by sentence
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
# Wordcloud is a technique to show which words are the most frequent in a given text
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# YouTube API key
api_key = 'YouTube API Key'

# List of Popular YouTube Channels for Kids
channel_ids = [
           'UCX6OQ3DkcsbYNE6H8uQQuVA'  # Mr Beast
            ]

# List of Playlist IDs for each channel
playlist_ids = [
             "UUX6OQ3DkcsbYNE6H8uQQuVA"
            ]


# Create the YouTube API client
youtube = build('youtube', 'v3', developerKey=api_key)

# Function to get video IDs from playlists
def get_video_ids(youtube, playlist_ids):
    all_video_ids = []

    for playlist_id in playlist_ids:
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50
        )
        response = request.execute()

        for item in response['items']:
            video_id = item['contentDetails']['videoId']
            all_video_ids.append(video_id)

    return all_video_ids

# Function to get channel stats
def get_channel_stats(youtube, channel_ids):
    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    for item in response['items']:
        data = {
            'channelName': item['snippet']['title'],
            'subscribers': item['statistics']['subscriberCount'],
            'views': item['statistics']['viewCount'],
            'totalVideos': item['statistics']['videoCount'],
            'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }

        all_data.append(data)

    return pd.DataFrame(all_data)

# Function to get video details
def get_video_details(youtube, video_ids):
    all_video_info = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {
                'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                'statistics': ['viewCount', 'likeCount', 'favoriteCount', 'commentCount'],
                'contentDetails': ['definition', 'caption']
            }
            video_info = {'video_id': video['id']}

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

# Get video IDs from playlists
video_ids = get_video_ids(youtube, playlist_ids)

# Get channel statistics
channel_df = get_channel_stats(youtube, channel_ids)

# Get video details
video_df = get_video_details(youtube, video_ids)

# Rename the 'channelTitle' column in video_df to 'channelName'
#This is done in order to merge the dataframes on a common column
video_df.rename(columns={'channelTitle': 'channelName'}, inplace=True)

# Merge the two dataframes based on a common column (e.g., channelName)
merged_df = pd.merge(channel_df, video_df, on='channelName')
new_df = merged_df.copy()


# Fill missing values in 'tags' with an empty string
#merged_df['tags'].fillna('', inplace=True)

# Fill missing values in 'commentCount' with zero
#merged_df['commentCount'].fillna(0, inplace=True)

# Check for missing values in the merged dataframe
#missing_values = merged_df.isnull().sum()
#print(missing_values)

new_df['subscribers'] = new_df['subscribers'].astype(float)
new_df['views'] = new_df['views'].astype(float)
new_df['totalVideos'] = new_df['totalVideos'].astype(float)

new_df['subscribers'] = new_df['subscribers'].apply(lambda x: '{:,}'.format(x))
new_df['views'] = new_df['views'].apply(lambda x: '{:,}'.format(x))
new_df['totalVideos'] = new_df['totalVideos'].apply(lambda x: '{:,}'.format(x))


# Clean the data if necessary
def merged_df(df):
    # Convert numeric columns to appropriate data types
    df['subscribers'] = pd.to_numeric(df['subscribers'])
    df['views'] = pd.to_numeric(df['views'])
    df['totalVideos'] = pd.to_numeric(df['totalVideos'])

    # Convert publishedAt to datetime
    df['publishedAt'] = pd.to_datetime(df['publishedAt'])
    df['duration'] = pd.to_datetime(df['duration'])

    # Remove unnecessary characters from tags and convert to lowercase
    df['tags'] = df['tags'].str.lower().str.replace(r'[^\w\s]', '')

    # Remove stopwords from description and title
    stop_words = set(stopwords.words('english'))
    df['description'] = df['description'].apply(remove_stopwords)
    df['title'] = df['title'].apply(remove_stopwords)


    return df

# Convert columns to float64
#non_numeric_subscribers = pd.to_numeric(new_df['subscribers'], errors='coerce').isna()
#non_numeric_views = pd.to_numeric(new_df['views'], errors='coerce').isna()
#non_numeric_subscribers = pd.to_numeric(new_df['totalVideos'], errors='coerce').isna()

#print(new_df.loc[non_numeric_subscribers, 'subscribers'])
#print(new_df.loc[non_numeric_views, 'views'])
#print(new_df.loc[non_numeric_subscribers, 'totalVideos'])

new_df['subscribers'] = new_df['subscribers'].str.replace(',', '').astype(float)
new_df['views'] = new_df['views'].str.replace(',', '').astype(float)
new_df['totalVideos'] = new_df['totalVideos'].str.replace(',', '').astype(float)




In [ ]:
print(new_df.dtypes)

channelName       object
subscribers      float64
views            float64
totalVideos      float64
playlistId        object
video_id          object
title             object
description       object
tags              object
publishedAt       object
viewCount         object
likeCount         object
favoriteCount     object
commentCount      object
definition        object
caption           object
dtype: object


In [ ]:
# Install sqlite3 module (if not already installed)
!pip install pysqlite3

# Import sqlite3 module
import sqlite3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.1-cp310-cp310-linux_x86_64.whl size=151103 sha256=c185547c3da7386a7aaed74af8eced282ae1623587e08b62957b49afc7c78774
  Stored in directory: /root/.cache/pip/wheels/ed/1d/da/4bb79a0c7885888c6be9e31dad7ebd3b47b023be60469fd74d
Successfully built pysqlite3


In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('your_database.db')

# Create a cursor object
cursor = conn.cursor()

# Define the CREATE TABLE query
#create_table_query = '''
#CREATE TABLE youtube_creators (
    #channelName TEXT,
    #subscribers REAL,
    #views REAL,
    #totalVideos REAL,
    #playlistId TEXT,
    #video_id TEXT,
    #title TEXT,
    #description TEXT,
    #tags TEXT,
    #publishedAt TEXT,
    #viewCount TEXT,
    #likeCount TEXT,
    #favoriteCount TEXT,
    #commentCount TEXT,
    #duration TEXT,
    #definition TEXT,
    #caption TEXT
#);
#'''

# Execute the CREATE TABLE query
#cursor.execute(create_table_query)


In [ ]:
# Convert tags from list to string
new_df['tags'] = new_df['tags'].apply(lambda x: ','.join(x) if isinstance(x, list) else '')

# Write the data to the SQLite table
new_df.to_sql('youtube_creators', conn, if_exists='append', index=False)



50

In [ ]:
# Execute a query to fetch data from the table
cursor = conn.cursor()
cursor.execute("SELECT * FROM youtube_creators LIMIT 10")
data = cursor.fetchall()

# Print the fetched data
for row in data:
    print(row)

('MrBeast', 173000000.0, 29527785603.0, 741.0, 'UUX6OQ3DkcsbYNE6H8uQQuVA', 'CP82DAKzL1U', 'I Traded My Car At a Red Light', '', '', '2023-07-15T17:00:01Z', '142027023', '8594904', '0', '17930', 'hd', 'false')
('MrBeast', 173000000.0, 29527785603.0, 741.0, 'UUX6OQ3DkcsbYNE6H8uQQuVA', 'fuhE6PYnRMc', 'Train Vs Giant Pit', "Go buy my merch at https://mrbeast.store and if you're looking to set up an online store, it should be on a .Store domain. Thanks for sponsoring this video!\n\nThank you to Warped Perception for helping with Jet Car https://www.youtube.com/@WarpedPerception\n\n Fireworks provided by https://americanwholesalefireworks.com/ \nand https://www.PyroStarentertainment.com/\n\nSUBSCRIBE OR I TAKE YOUR DOG\n╔═╦╗╔╦╗╔═╦═╦╦╦╦╗╔═╗\n║╚╣║║║╚╣╚╣╔╣╔╣║╚╣═╣ \n╠╗║╚╝║║╠╗║╚╣║║║║║═╣\n╚═╩══╩═╩═╩═╩╝╚╩═╩═╝\n\n----------------------------------------------------------------\nfollow all of these or i will kick you\n• Facebook - https://www.facebook.com/MrBeast6000/\n• Twitter - https://twitter.com

In [ ]:
# Execute a query to fetch the list of tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the results
tables = cursor.fetchall()

# Print the tables
for table in tables:
    print(table[0])


youtube_creators


In [ ]:
# Execute a query to fetch the table's schema or column names
cursor.execute("PRAGMA table_info(youtube_creators)")

# Fetch all the column names
columns = [column[1] for column in cursor.fetchall()]

# Print the column names
print(columns)

['channelName', 'subscribers', 'views', 'totalVideos', 'playlistId', 'video_id', 'title', 'description', 'tags', 'publishedAt', 'viewCount', 'likeCount', 'favoriteCount', 'commentCount', 'definition', 'caption']


In [ ]:
#Channel Performance Analysis:

# Query: Which channel has the highest number of subscribers?
query1 = "SELECT channelName, subscribers FROM youtube_creators ORDER BY subscribers DESC LIMIT 1"
cursor.execute(query1)
result1 = cursor.fetchall()
print("Channel with the highest number of subscribers:", result1)

# Query: Which channel has the highest number of views?
query2 = "SELECT channelName, views FROM youtube_creators ORDER BY views DESC LIMIT 1"
cursor.execute(query2)
result2 = cursor.fetchall()
print("Channel with the highest number of views:", result2)

# Query: Which channel has the highest number of videos?
query3 = "SELECT channelName, totalVideos FROM youtube_creators ORDER BY totalVideos DESC LIMIT 1"
cursor.execute(query3)
result3 = cursor.fetchall()
print("Channel with the highest number of videos:", result3)



Channel with the highest number of subscribers: [('MrBeast', 173000000.0)]
Channel with the highest number of views: [('MrBeast', 29527785603.0)]
Channel with the highest number of videos: [('MrBeast', 741.0)]


In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('your_database.db')
cursor = conn.cursor()

# Execute the SQL query
query = """
    SELECT channelName, title, viewCount
    FROM youtube_creators
    ORDER BY channelName, viewCount DESC
"""

cursor.execute(query)

# Fetch all the rows from the result set
rows = cursor.fetchall()

# Create a dictionary to store the top 5 videos for each YouTuber
top_videos_data = {}

# Process the rows and populate the dictionary
for row in rows:
    channel_name = row[0]
    title = row[1]
    view_count = row[2]

    if channel_name not in top_videos_data:
        top_videos_data[channel_name] = []

    top_videos_data[channel_name].append({'title': title, 'views': view_count})

# Close the database connection
conn.close()

# Print the top 5 videos for each YouTuber
for channel_name, videos in top_videos_data.items():
    print(f"Channel: {channel_name}")
    for video in videos[:5]:
        print(f"Title: {video['title']}, Views: {video['views']}")
    print()


Channel: MrBeast
Title: Extreme $1,000,000 Hide And Seek, Views: 93348399
Title: I Didn’t Eat Food For 30 Days, Views: 88361239
Title: Would You Fly To Paris For A Baguette?, Views: 866648839
Title: Giving iPhones Instead Of Candy on Halloween, Views: 789053135
Title: $1,000,000 Influencer Tournament!, Views: 77748426

